In [1]:
#from sklearn import linear_model
from torch import nn

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import torch  as tc

In [2]:
class SineActivation(nn.Module):
    def __init__(self): 
        super(SineActivation, self).__init__() 
    def forward(self, x):
        return tc.sin(x)
    
class Regressao_M(nn.Module):
    def __init__(self, neuronio, M, output=1, activation=nn.Mish(), creat_p=False, N_of_paramater=1):
        super().__init__()
        self.neuronio = neuronio
        self.M = M
        self.output = output
        self.creat_p = creat_p
        self.N_of_paramater = N_of_paramater

        # Lista para armazenar as camadas lineares
        self.hidden_layers = nn.ModuleList([nn.Linear(1, neuronio)])
        self.hidden_layers.extend([nn.Linear(neuronio, neuronio) for _ in range(M-1)])
        
        # Última camada linear
        self.output_layer = nn.Linear(neuronio, output)

        # Função de ativação
        self.activation = activation

        if creat_p:
            self.acceleration = nn.Parameter(tc.randn(N_of_paramater))

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
        x = self.output_layer(x)
        return x


# Oscilador harmonico Simples

$$

\frac{{d^2x}}{{dt^2}} + \omega_0^2 x =0

$$

$ \omega_0 = \sqrt{g/l}$ é a frequencia de oscilação em termo de g = gravidade da mola e l=comprimento do pendulo. 

Solução considerando $ \omega(0)=0$ e $ \theta(0)=\theta_0$:


$$
\theta(t) = \theta_0cos(w_0t) 
$$

In [3]:
def soluçao_OHM(x_0,omega_0,t):
    x_0_ = tc.tensor(x_0)
    return x_0_*tc.cos(tc.tensor(omega_0)*t)

### Aprendendo a função sabendo os parametros

In [4]:
x_0 = np.pi/4
x_i = 0
x_f = 4 
Omega_0 = np.sqrt(980/30)
print(Omega_0)

# Criando os dados de treino
t_train = tc.linspace( x_i, x_f, 100, requires_grad = True).reshape(-1,1)
y_train = soluçao_OHM( x_0 = x_0, omega_0 = Omega_0 , t = t_train)

omega_data = y_train

model = Regressao_M(neuronio = 15,M=2,output=2, activation=SineActivation())
opt = tc.optim.Adam(params=model.parameters(),lr=0.001)



5.715476066494082


In [5]:
LOSS = []
for epoch in range(2000):
    y_preds = model(t_train)
    solution_theta = y_preds.reshape(-1,1)

    ###########################################################################
    d_theta_dt = tc.autograd.grad(solution_theta,
                                    t_train, 
                                    grad_outputs = tc.ones_like(solution_theta),
                                    create_graph = True)[0]
    d_omega_dt = tc.autograd.grad(d_theta_dt,
                                    t_train, 
                                    grad_outputs = tc.ones_like(d_theta_dt),
                                    create_graph=True)[0]

    # Edo
    loss_ode_dxdt = tc.mean( (d_theta_dt -omega_data)**2 )
    loss_ode_dvdt = tc.mean( (d_omega_dt + Omega_0**2*solution_theta )**2 )

    loss = loss_ode_dvdt +loss_ode_dxdt
    ############################################################################

    opt.zero_grad()
    loss.backward(retain_graph=True)
    opt.step()
    
    LOSS.append(loss.cpu().detach().numpy())

plt.plot(LOSS)
plt.legend()
plt.yscale('log')
plt.show()


RuntimeError: The size of tensor a (100) must match the size of tensor b (200) at non-singleton dimension 0

In [ ]:
t_test = tc.linspace(x_i,x_f,200,requires_grad=True).reshape(-1,1)
y_test = soluçao_OHM(x_0 = x_0 ,omega_0 = Omega_0 ,t = t_test)

In [ ]:
y_preds_test = model(t_test)
solution_x = y_preds_test[:,0].reshape(-1,1)
solution_v = y_preds_test[:,1].reshape(-1,1)

fig, ax = plt.subplots(1,2,figsize=(10,4))
ax[0].plot(t_test.detach().numpy(), y_test.detach().numpy(),"k-", label="teorico")
ax[0].plot(t_test.detach().numpy(), solution_x.detach().numpy(),"r--", label=f"simulação x" )
ax[0].legend()

ax[1].plot(t_test.detach().numpy(), v_test.detach().numpy(),"k-", label="teorico")
ax[1].plot(t_test.detach().numpy(), solution_v.detach().numpy(),"g--", label=f"simulação v" )
ax[1].legend()
plt.show()